In [8]:
import numpy as np
import pandas as pd

In [9]:
df = pd.read_csv('transformed_data.csv')
df.head()

,Total_Stops,Price,day,month,Duration_min,Source_Banglore,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,...,Additional_Info_1 Long layover,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,0,3897.0,24,3,170,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,2,7662.0,1,5,445,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
2,2,13882.0,9,6,1140,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6218.0,12,5,325,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
4,1,13302.0,1,3,285,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10462 entries, 0 to 10461
Data columns (total 38 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Total_Stops                                   10462 non-null  int64  
 1   Price                                         10462 non-null  float64
 2   day                                           10462 non-null  int64  
 3   month                                         10462 non-null  int64  
 4   Duration_min                                  10462 non-null  int64  
 5   Source_Banglore                               10462 non-null  bool   
 6   Source_Chennai                                10462 non-null  bool   
 7   Source_Delhi                                  10462 non-null  bool   
 8   Source_Kolkata                                10462 non-null  bool   
 9   Source_Mumbai                                 10462 non-null 

In [11]:
from sklearn.model_selection import train_test_split

X = df.drop('Price', axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(8369, 37) (8369,) (2093, 37) (2093,)


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [13]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [14]:
models = [
    ['LinearRegression : ', LinearRegression()],
    ['Lasso : ', Lasso()],
    ['Ridge : ', Ridge()],
    ['ElasticNet :', ElasticNet()],
    ['KNeighborsRegressor : ', KNeighborsRegressor()],
    ['DecisionTreeRegressor : ', DecisionTreeRegressor()],
    ['RandomForestRegressor : ', RandomForestRegressor()],
    ['GradientBoostingRegressor : ', GradientBoostingRegressor()],
    ['XGBRegressor : ', XGBRegressor()],
    ['SVR : ', SVR()],
    ['AdaBoostRegressor : ', AdaBoostRegressor()]
]

In [15]:
for name, model in models:
    model=model
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(name, (np.sqrt(r2_score(y_test, predictions))))

LinearRegression :  0.8130299300383194


c:\Projects\MLOps\Flight-Price\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.492e+07, tolerance: 1.801e+07
  model = cd_fast.enet_coordinate_descent(


Lasso :  0.812347064140187
Ridge :  0.8129109820760152
ElasticNet : 0.6746990753922234
KNeighborsRegressor :  0.7192169403502
DecisionTreeRegressor :  0.8665119630248186
RandomForestRegressor :  0.916226188861264
GradientBoostingRegressor :  0.8940986680376635
XGBRegressor :  0.9251438879781091
SVR :  0.4052150172846858
AdaBoostRegressor :  0.7221389283287186


In [16]:
algorithms = {
    'XGBRegressor' : {
        'model' : XGBRegressor(),
        'param' : {
            'learning_rate' : [0.1, 0.3, 0.5, 0.25],
            'max_depth' : [3, 5, 7],
            'gamma' : [0.1, 0.3, 0.5],
            'min_child_weight' : [3, 5, 7, 9],
        }
    },
    'RandomForestRegressor' : {
        'model' : RandomForestRegressor(),
        'param' : {
            'n_estimators' : [1000, 300, 500],
            'max_depth' : [9, 5, 7],
            'min_samples_split' : [2, 4, 6, 8]
        }
    },
    'GradientBoostingRegressor' : {
        'model' : GradientBoostingRegressor(),
        'param' : {
            'learning_rate' : [0.5, 0.1, 0.20, 0.30],
            'n_estimators' : [300, 500, 1000],
        }
    }
}

from sklearn.model_selection import GridSearchCV

In [17]:
score = []

for name, mp in algorithms.items() :
    rs = GridSearchCV(estimator = mp['model'], param_grid = mp['param'], cv = 10, n_jobs=-1, verbose=3)
    rs.fit(X_train, y_train)
    score.append({
        'model': name,
        'score' : rs.best_score_,
        'params' : rs.best_params_
    })

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


c:\Projects\MLOps\Flight-Price\venv\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 10 folds for each of 36 candidates, totalling 360 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


In [18]:
final = pd.DataFrame(score, columns=['model', 'score', 'params'])
final

,model,score,params
0,XGBRegressor,0.865866,"{'gamma': 0.1, 'learning_rate': 0.3, 'max_dept..."
1,RandomForestRegressor,0.837372,"{'max_depth': 9, 'min_samples_split': 2, 'n_es..."
2,GradientBoostingRegressor,0.861548,"{'learning_rate': 0.2, 'n_estimators': 1000}"


In [21]:
for m in range(len(final)):
    for i in final.iloc[m, :]:
        print(i)

XGBRegressor
0.865865623224056
{'gamma': 0.1, 'learning_rate': 0.3, 'max_depth': 5, 'min_child_weight': 3}
RandomForestRegressor
0.8373721924363211
{'max_depth': 9, 'min_samples_split': 2, 'n_estimators': 1000}
GradientBoostingRegressor
0.8615477299156321
{'learning_rate': 0.2, 'n_estimators': 1000}
